In [1]:
import pandas as pd
import numpy as np
import os as os
#import pickle
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, StratifiedKFold, KFold, RepeatedKFold
from sklearn.metrics import classification_report
from joblib import dump, load
import acessos as ac
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

QUERY 

DIA MÍNIMO E MÁXIMO DE INGESTÃO 
- 2023-09-29 16:47:45.401
- 2023-11-24 02:20:19.067

In [2]:

query = """select 
id_conversation
, ingestion_date
, ds_comentario
from processed_zone_events_intercom.issues_ratings_intercom
where ds_comentario is not null"""

df = ac.df_athena_q('flavia-costa', query)

In [3]:
df.head()

,id_conversation,ingestion_date,ds_comentario
0,399047,2023-09-29 16:47:45.401,ótimo
1,400910,2023-09-29 16:47:45.401,excelente atendimento luiza❤
2,400624,2023-09-29 16:47:45.401,como que eu faço para voltar meu cartão
3,400771,2023-09-29 16:47:45.401,bom
4,400884,2023-09-29 16:47:45.401,ótimo atendimento.


In [4]:
len(df)

42870

## Aplicando modelo sentimento

In [5]:
from sklearn.neural_network import MLPClassifier


vetorizador = load('vetorizador_sentimento.joblib') 

In [7]:
X = vetorizador.transform(df['ds_comentario'])

In [8]:
clf_sentiment = load('modelo_sentimento.joblib') 

In [9]:
df['sent_predito_1'] = clf_sentiment.predict(X)

### Palavras chave 

In [10]:
chave_negativo = ['falta de respeito', 
'judicial',
'procon',
'jurídico',
'porra',
'merda', 
'palhaçada',
'carai',
'caralho',
'bosta',
'vocês demoram',
'voces demoram',
'vcs demoram',
'horrivel',
'banco ruim',
'péssimo',
'danos morais',
'pelo amor de deus',
'eu preciso resolver',
'atender??',
'me roubando']


não fui respondid
não tive resposta
não resolveu o meu problema
meu problema ainda não foi resolvido
não ajudou

In [11]:
chave_positivo = [
'ótimo',
'otimo',
'perfeito',
'muito obrigada',
'excelente',
'ótimo dia',
'maravilha',
'deus abençoe']

Acrescentar:

Atendimento maravilhoso
atendimento nota 10
ótimo atendimento
atendimento excelente

In [12]:
df['sent_predito_1'].value_counts()

0    18373
2    12386
1    12111
Name: sent_predito_1, dtype: int64

In [14]:
def marca_negativo(x):
    for i in chave_negativo:
        if (x.find(i) != -1):
            return 1
        else:
            return 0
   
df['flag_negativo'] = df['ds_comentario'].apply(marca_negativo)

In [15]:
def marca_positivo(x):
    for i in chave_positivo:
        if (x.find(i) != -1):
            return 1
        else:
            return 0
   
df['flag_positivo'] = df['ds_comentario'].apply(marca_positivo)

Para a pesquisa do CSAT temos uma tendencia maior para comentários positivos portanto podemos alterar a regra, que está diferente para análise de sentimento do chat

In [16]:
df['sent_predito_final'] = np.where((df['flag_negativo'] == 1),  0, #condições negativo
                                 np.where(df['flag_positivo'] == 1,  2, #condições positivo
                                          df['sent_predito_1']))

df['sent_predito_final'].value_counts()

0    18375
2    12480
1    12015
Name: sent_predito_final, dtype: int64

### Modelo do tipo de mensagem

Gostaria de saber
Como faço
como faço
problema
não reconheço
não fiz essa compra
não reconhecida
fui roubad
fui assaltad
limite não voltou
não consegui pagar
pix não chegou
compra que não fiz
nunca comprei
quero alterar meu
quero resolver
queria que fosse resolvido
preciso resolver
quero aumentar o meu limite
queria o aumento do meu cartao
fiquei na dúvida
paguei mas não aparece
prefiro parcelar

In [23]:
clf_tipo_msg = load('modelo_tipo_mensagem.joblib')

In [24]:
df['tipo_msg_predito'] = clf_tipo_msg.predict(X)

In [25]:
df['tipo_msg_predito'].value_counts()

0    394796
2    256753
1    201334
Name: tipo_msg_predito, dtype: int64

In [80]:
df['sentimento_predito'] = np.where(df['sent_predito_final'] == 0,  '0. Negativo', 
                           np.where(df['sent_predito_final'] == 2,  '2. Positivo', '1. Neutro'))

In [42]:
df.query('sentimento_predito == "0. Negativo"')['ds_message'][27]

'[Dúvida sobre uma compra, Apareceu uma compra que eu não fiz, Pela 5° x, Eu não vou mais falar com vcs, Vcs demora 2 horas pra responder quando responde eu não vejo pq não sou vidente, Já mandei todas as informações q vcs me pedem, Estão me fazendo de otário, Ou vcs resolvem minha situação ou irei tomar medidas judiciais, 02/09/2023 APPLE.COM/BILL SAO PAULO BRA 54,90\n02/09/2023 APPLE.COM/BILL SAO PAULO BRA 54,90\n02/09/2023 APPLE.COM/BILL SAO PAULO BRA 27,90\n02/09/2023 APPLE.COM/BILL SAO PAULO BRA 54,90\n02/09/2023 APPLE.COM/BILL SAO PAULO BRA 27,90\n02/09/2023 APPLE.COM/BILL SAO PAULO BRA 4,90\n02/09/2023 APPLE.COM/BILL SAO PAULO BRA 4,90\n27/08/2023 APPLE.COM/BILL SAO PAULO BRA 4,90, Parem de me perguntar se tem outras compras q nao foi eu q comprei q estão na minha time LINE. As únicas q não foi eu são justamente essas q te mandei, Pela 5 x, Pare de me enrolar porra, Já tô ficando estressado, Caramba rapaz, Vcs são incompetentes, Todos vcs, Sem excessão, Resolva minha questão, Ag

In [46]:
df['tipo_msg_predito'] = np.where(df['tipo_msg_predito'] == 0,  '0. Problema', 
                           np.where(df['tipo_msg_predito'] == 1,  '1. Solicitacao', '2. Duvida'))

### Avaliação final e exportação

In [17]:
df.head()

,id_conversation,ingestion_date,ds_comentario,sent_predito_1,flag_negativo,flag_positivo,sent_predito_final
0,399047,2023-09-29 16:47:45.401,ótimo,2,0,1,2
1,400910,2023-09-29 16:47:45.401,excelente atendimento luiza❤,0,0,0,0
2,400624,2023-09-29 16:47:45.401,como que eu faço para voltar meu cartão,1,0,0,1
3,400771,2023-09-29 16:47:45.401,bom,2,0,0,2
4,400884,2023-09-29 16:47:45.401,ótimo atendimento.,0,0,1,0


In [18]:
df.to_csv('sentimento_csat.csv', index = False)

In [19]:
query2 = """select 
id_conversation, 
ds_nota
from processed_zone_events_intercom.issues_ratings_intercom
where ds_comentario is not null"""

df_nota = ac.df_athena_q('flavia-costa', query2)

In [20]:
df_nota.head()

,id_conversation,ds_nota
0,222491,😠
1,268393,1 star
2,248757,4 stars
3,270286,😍
4,269810,😍


In [21]:
df = df.join(df_nota.set_index('id_conversation'), on = 'id_conversation', how = 'left')

In [22]:
df.head()

,id_conversation,ingestion_date,ds_comentario,sent_predito_1,flag_negativo,flag_positivo,sent_predito_final,ds_nota
0,399047,2023-09-29 16:47:45.401,ótimo,2,0,1,2,😃
1,400910,2023-09-29 16:47:45.401,excelente atendimento luiza❤,0,0,0,0,😍
2,400624,2023-09-29 16:47:45.401,como que eu faço para voltar meu cartão,1,0,0,1,🙁
3,400771,2023-09-29 16:47:45.401,bom,2,0,0,2,😃
4,400884,2023-09-29 16:47:45.401,ótimo atendimento.,0,0,1,0,😍


In [35]:
df.query('sent_predito_1 == 0').head()

,id_conversation,ingestion_date,ds_comentario,sent_predito_1,flag_negativo,flag_positivo,sent_predito_final,ds_nota
1,400910,2023-09-29 16:47:45.401,excelente atendimento luiza❤,0,0,0,0,😍
4,400884,2023-09-29 16:47:45.401,ótimo atendimento.,0,0,1,0,😍
8,348052,2023-09-29 16:47:45.401,amei o atendimento !,0,0,0,0,😍
9,347268,2023-09-29 16:47:45.401,demora muito pra responder o cliente,0,0,0,0,🙁
10,347765,2023-09-29 16:47:45.401,uma demora danada e quando entrão no chat esse...,0,0,0,0,😠


In [28]:
pd.crosstab(df.ds_nota, df.sent_predito_final, normalize = 'index')

sent_predito_final,0,1,2
ds_nota,,,
1 star,0.634228,0.306488,0.059284
2 stars,0.404624,0.450867,0.144509
3 stars,0.435754,0.413408,0.150838
4 stars,0.307420,0.261484,0.431095
5 stars,0.274716,0.195101,0.530184
😃,0.256804,0.277794,0.465402
😍,0.303929,0.228062,0.468009
😐,0.389506,0.460802,0.149691
😠,0.684403,0.264466,0.051131


In [84]:
""" area = 'cx'
name = 'analise_sentimento_teste.csv'
filename = f'{area}.{name}'

df[['cpf_customer', 'mes_chat', 'dt_envio_mensagem',
       'ds_message', 'sent_predito_1','tipo_msg_predito', 'sentimento_predito']].to_csv(filename, index=False)

os.system(f'aws s3 cp {filename} s3://data-sandbox-zone-will-prod/upload/') """

upload: ./cx.analise_sentimento_teste.csv to s3://data-sandbox-zone-will-prod/upload/cx.analise_sentimento_teste.csv


0